In [4]:
!conda list --export > requirements.txt

In [ ]:
!pip install conda-forge::streamlit -y
!pip install anaconda::joblib -y


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -y

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -y


In [63]:
!pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB 487.6 kB/s eta 0:05:08
   ---------------------------------------- 0.1/150.0 MB 558.5 kB/s eta 0:04:29
   ---------------------------------------- 0.1/150.0 MB 508.4 kB/s eta 0:04:55
   ---------------------------------------- 0.1/150.0 MB 552.2 kB/s eta 0:04:32
   ---------------------------------------- 0.1/150.0 MB 532.5 kB/s eta 0:04:42
   ---------------------------------------- 0.2/150.0 MB 620.6 kB/s eta 0:04:02
   ---------------------------------------- 0.2/150.0 MB 653.6 kB/s eta 0:03:50
   ---------------------------------------- 0.3/150.0 MB 679.3 kB/s eta 0:03:41
   ---------------------------------------- 0.3/150.0 MB 728.0 kB/s eta 0:03:26
   ---------------------------------------- 0.4/150.0 MB 742.3 kB/s eta 0:03:22
   ---------------------------------------- 0.4/150.0 MB 7

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import joblib
import streamlit as st

In [68]:


# Download latest version
df = pd.read_csv('2nd_try_cleaned_Data.csv')
df = df.drop("Unnamed: 0", axis=1)
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,0,16.60,1,0,0,3,30,0,0,7,0,1,1,3,5,1,0,1
1,0,20.34,0,0,1,0,0,0,0,12,0,0,1,3,7,0,0,0
2,0,26.58,1,0,0,20,30,0,1,9,0,1,1,1,8,1,0,0
3,0,24.21,0,0,0,0,0,0,0,11,0,0,0,2,6,0,0,1
4,0,23.71,0,0,0,28,0,1,0,4,0,0,1,3,8,0,0,0


In [69]:
print("=== Initial Exploration ===")
print("Shape:", df.shape)
print("\nFirst 5 rows:")
display(df.head())
print("\nInfo:")
df.info()
print("\nSummary stats:")
display(df.describe(include='all'))

=== Initial Exploration ===
Shape: (69080, 18)

First 5 rows:


,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,0,16.60,1,0,0,3,30,0,0,7,0,1,1,3,5,1,0,1
1,0,20.34,0,0,1,0,0,0,0,12,0,0,1,3,7,0,0,0
2,0,26.58,1,0,0,20,30,0,1,9,0,1,1,1,8,1,0,0
3,0,24.21,0,0,0,0,0,0,0,11,0,0,0,2,6,0,0,1
4,0,23.71,0,0,0,28,0,1,0,4,0,0,1,3,8,0,0,0



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69080 entries, 0 to 69079
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   HeartDisease      69080 non-null  int64  
 1   BMI               69080 non-null  float64
 2   Smoking           69080 non-null  int64  
 3   AlcoholDrinking   69080 non-null  int64  
 4   Stroke            69080 non-null  int64  
 5   PhysicalHealth    69080 non-null  int64  
 6   MentalHealth      69080 non-null  int64  
 7   DiffWalking       69080 non-null  int64  
 8   Sex               69080 non-null  int64  
 9   AgeCategory       69080 non-null  int64  
 10  Race              69080 non-null  int64  
 11  Diabetic          69080 non-null  int64  
 12  PhysicalActivity  69080 non-null  int64  
 13  GenHealth         69080 non-null  int64  
 14  SleepTime         69080 non-null  int64  
 15  Asthma            69080 non-null  int64  
 16  KidneyDisease     69080 non-null 

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
count,69080.000000,69080.000000,69080.000000,69080.000000,69080.000000,69080.000000,69080.000000,69080.000000,69080.000000,69080.000000,69080.000000,69080.000000,69080.000000,69080.000000,69080.000000,69080.000000,69080.000000,69080.000000
mean,0.086812,28.002076,0.420860,0.069774,0.040446,3.567979,4.005515,0.145585,0.468891,6.589186,0.950391,0.136740,0.772496,2.573321,7.098017,0.139751,0.039433,0.108150
std,0.281562,6.319090,0.493701,0.254768,0.197004,8.133333,8.042792,0.352692,0.499035,3.588422,1.624391,0.343575,0.419224,1.063582,1.511295,0.346731,0.194623,0.310572
min,0.000000,12.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,23.670000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000
50%,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000,0.000000,1.000000,3.000000,7.000000,0.000000,0.000000,0.000000
75%,0.000000,31.140000,1.000000,0.000000,0.000000,2.000000,4.000000,0.000000,1.000000,10.000000,1.000000,0.000000,1.000000,3.000000,8.000000,0.000000,0.000000,0.000000
max,1.000000,87.050000,1.000000,1.000000,1.000000,30.000000,30.000000,1.000000,1.000000,12.000000,5.000000,1.000000,1.000000,4.000000,24.000000,1.000000,1.000000,1.000000


In [70]:
df.isna().sum()

HeartDisease        0
BMI                 0
Smoking             0
AlcoholDrinking     0
Stroke              0
PhysicalHealth      0
MentalHealth        0
DiffWalking         0
Sex                 0
AgeCategory         0
Race                0
Diabetic            0
PhysicalActivity    0
GenHealth           0
SleepTime           0
Asthma              0
KidneyDisease       0
SkinCancer          0
dtype: int64

In [71]:
df_numeric = df.select_dtypes(include='number')

print(df_numeric.head())  # أول 5 صفوف من الأعمدة الرقمية

   HeartDisease    BMI  Smoking  AlcoholDrinking  Stroke  PhysicalHealth  \
0             0  16.60        1                0       0               3   
1             0  20.34        0                0       1               0   
2             0  26.58        1                0       0              20   
3             0  24.21        0                0       0               0   
4             0  23.71        0                0       0              28   

   MentalHealth  DiffWalking  Sex  AgeCategory  Race  Diabetic  \
0            30            0    0            7     0         1   
1             0            0    0           12     0         0   
2            30            0    1            9     0         1   
3             0            0    0           11     0         0   
4             0            1    0            4     0         0   

   PhysicalActivity  GenHealth  SleepTime  Asthma  KidneyDisease  SkinCancer  
0                 1          3          5       1              0   

In [72]:
# Create a 2x2 subplot grid with Plotly
fig = make_subplots(rows=3, cols=2, 
                    subplot_titles=('Survival Distribution', 
                                    'Age Distribution', 
                                    'smoking_count', 
                                    'Correlation Matrix',
                                    'smokers and sick'),
                    vertical_spacing=0.15,  # Add spacing to avoid overlap
                    horizontal_spacing=0.15)

# Survival count plot
survived_counts = df['HeartDisease'].value_counts()
# Add "Did not survive" bar
fig.add_trace(
    go.Bar(
        x=['sick'], 
        y=[survived_counts.get(0, 0)], 
        name='sick',  # Explicit legend label
        marker=dict(color='red'),  # Red for "Did not survive"
        text=[survived_counts.get(0, 0)], 
        textposition='auto'
    ),
    row=1, col=1
)

# Add "Survived" bar
fig.add_trace(
    go.Bar(
        x=['well'], 
        y=[survived_counts.get(1, 0)], 
        name='well',  # Explicit legend label
        marker=dict(color='blue'),  # Blue for "Survived"
        text=[survived_counts.get(1, 0)], 
        textposition='auto'
    ),
    row=1, col=1
)

# Age distribution with histogram and KDE
age = pd.to_numeric(df['AgeCategory'], errors='coerce')
age = age.dropna()
age = df['AgeCategory']
kde = gaussian_kde(age)
x = np.linspace(age.min(), age.max(), 12)
kde_values = kde(x)
fig.add_trace(
    go.Histogram(x=age, 
                nbinsx=20, 
                histnorm='probability density', 
                name='AgeCategory', 
                marker=dict(color='orange'),  # Set histogram color to orange
                opacity=0.7  # Add slight transparency to see KDE line
                ),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=x, y=kde_values, mode='lines', name='KDE', line=dict(color='red')),
    row=1, col=2
)


smoking_count = df['Smoking'].value_counts()
# Add "smokers" bar
fig.add_trace(
    go.Bar(
        x=['YES'], 
        y=[smoking_count.get(0, 0)], 
        name='YES',  # Explicit legend label
        marker=dict(color='red'),  # Red for "Did not survive"
        text=[smoking_count.get(0, 0)], 
        textposition='auto'
    ),
    row=2, col=1
)

# Add "NO smokers" bar
fig.add_trace(
    go.Bar(
        x=['NO'], 
        y=[smoking_count.get(1, 0)], 
        name='NO',  # Explicit legend label
        marker=dict(color='blue'),  # Blue for "Survived"
        text=[smoking_count.get(1, 0)], 
        textposition='auto'
    ),
    row=2, col=1
)


# Correlation heatmap with adjusted colorbar
corr_matrixx = df.corr()
corr_matrix=corr_matrixx[['Stroke','AlcoholDrinking','Smoking']]
fig.add_trace(
    go.Heatmap(
        z=corr_matrix.values, 
        x=corr_matrix.columns, 
        y=corr_matrix.columns, 
        colorscale='RdBu',
        text=corr_matrix.values.round(2),  # Add correlation values rounded to 2 decimal places
        texttemplate="%{text}",  # Display the values in each cell
        textfont={"size": 10},  # Set font size for readability
        colorbar=dict(
            len=0.45,  # Set length to match the subplot height (0.4 is roughly the height of one subplot in a 2x2 grid)
            y=0.21,    # Position the colorbar to align with the bottom subplot (row 2)
            yanchor='middle'  # Center the colorbar vertically within the subplot
        )
    ),
    row=2, col=2
)

grouped = df.groupby(['Smoking', 'HeartDisease']).size().unstack(fill_value=0)


fig.add_trace(
    go.Bar(
        name='Sick = 1',
        x=grouped.index,
        y=grouped[1],
        marker_color='red'
    ),    row=3, col=1
)

fig.add_trace(
    go.Bar(
        name='Sick = 0',
        x=grouped.index,
        y=grouped[0],
        marker_color='blue'
    ),    row=3, col=1

)

# Update layout and axes
fig.update_layout(
    height=1200, 
    width=1200, 
    title_text="Univariate and Multivariate Analysis",
    title_x=0.5,  # Center the title
    showlegend=True,
    barmode='group'  # Group the bars for Survival Distribution
)

# Add axis labels for clarity
# fig.update_xaxes(title_text="Survival Status", row=1, col=1)
# fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_xaxes(title_text="Age", row=1, col=2)
fig.update_yaxes(title_text="Probability Density", row=1, col=2)
fig.update_xaxes(title_text="smoking count", row=2, col=1)

fig.show()

In [73]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Separate data into two groups: Heart Disease (1) and Non-Heart Disease (0)
# heart_disease = df[df['HeartDisease'] == 1]
# non_heart_disease = df[df['HeartDisease'] == 0]

# # Split each group separately (70% train, 30% test)
# X_heart_train, X_heart_test, y_heart_train, y_heart_test = train_test_split(
#     heart_disease.drop(columns=['HeartDisease']), heart_disease['HeartDisease'], test_size=0.3, random_state=42)

# X_non_heart_train, X_non_heart_test, y_non_heart_train, y_non_heart_test = train_test_split(
#     non_heart_disease.drop(columns=['HeartDisease']), non_heart_disease['HeartDisease'], test_size=0.3, random_state=42)

# # Combine the separate train and test sets
# X_train = pd.concat([X_heart_train, X_non_heart_train], axis=0)
# y_train = pd.concat([y_heart_train, y_non_heart_train], axis=0)

# X_test = pd.concat([X_heart_test, X_non_heart_test], axis=0)
# y_test = pd.concat([y_heart_test, y_non_heart_test], axis=0)

# # Print class distribution after splitting
# print("Train set distribution:\n", y_train.value_counts())
# print("\nTest set distribution:\n", y_test.value_counts())

# # Print shapes
# print("\nX_train Shape:", X_train.shape, "y_train Shape:", y_train.shape)
# print("X_test Shape:", X_test.shape, "y_test Shape:", y_test.shape)


In [76]:
# Define features and target
X = df.drop('HeartDisease', axis=1)  # شيل العمود الهدف
y = df['HeartDisease'] 
# BMI	Smoking	Smoking	Stroke	PhysicalHealth	MentalHealth	DiffWalking	Sex	AgeCategory	Race	Diabetic	PhysicalActivity	GenHealth	SleepTime	Asthma	KidneyDisease	SkinCancer
# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['BMI', 'Smoking', 'AlcoholDrinking','Stroke','PhysicalHealth','MentalHealth','DiffWalking','Sex','AgeCategory',
                                   'Race',	'Diabetic',	'PhysicalActivity',	'GenHealth',	'SleepTime',	'Asthma',	'KidneyDisease',	'SkinCancer'])
    ])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [81]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the model
xgb = XGBClassifier(n_estimators=100, max_depth=10, random_state=42, use_label_encoder=False, eval_metric="logloss")

# Train the model
xgb.fit(preprocessor.fit_transform(X_train), y_train)
# Make predictions
y_pred = xgb.predict(preprocessor.transform(X_test))

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


c:\Users\ahmed\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning:

[01:36:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.




Accuracy: 0.9101042269832079
              precision    recall  f1-score   support

           0       0.93      0.98      0.95     12639
           1       0.43      0.16      0.23      1177

    accuracy                           0.91     13816
   macro avg       0.68      0.57      0.59     13816
weighted avg       0.88      0.91      0.89     13816



In [82]:
print("\n=== Model Performance ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


=== Model Performance ===
Accuracy: 0.91

Confusion Matrix:
[[12388   251]
 [  991   186]]

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.98      0.95     12639
           1       0.43      0.16      0.23      1177

    accuracy                           0.91     13816
   macro avg       0.68      0.57      0.59     13816
weighted avg       0.88      0.91      0.89     13816



In [83]:
joblib.dump(xgb, 'project_model.pkl')
joblib.dump(preprocessor, 'preprocessor11.pkl')

['preprocessor11.pkl']

In [84]:
%%writefile st_app_55.py

import streamlit as st
import joblib
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde

# Cache the data loading and figure creation for performance
@st.cache_data
def load_data_and_create_figure():
    # Load the Titanic dataset
    df = pd.read_csv('2nd_try_cleaned_Data.csv')

    # Create a 2x2 subplot grid with Plotly
fig = make_subplots(rows=3, cols=2, 
                    subplot_titles=('Survival Distribution', 
                                    'Age Distribution', 
                                    'smoking_count', 
                                    'Correlation Matrix',
                                    'smokers and sick'),
                    vertical_spacing=0.15,  # Add spacing to avoid overlap
                    horizontal_spacing=0.15)

# Survival count plot
survived_counts = df['HeartDisease'].value_counts()
# Add "Did not survive" bar
fig.add_trace(
    go.Bar(
        x=['sick'], 
        y=[survived_counts.get(0, 0)], 
        name='sick',  # Explicit legend label
        marker=dict(color='red'),  # Red for "Did not survive"
        text=[survived_counts.get(0, 0)], 
        textposition='auto'
    ),
    row=1, col=1
)

# Add "Survived" bar
fig.add_trace(
    go.Bar(
        x=['well'], 
        y=[survived_counts.get(1, 0)], 
        name='well',  # Explicit legend label
        marker=dict(color='blue'),  # Blue for "Survived"
        text=[survived_counts.get(1, 0)], 
        textposition='auto'
    ),
    row=1, col=1
)

# Age distribution with histogram and KDE
age = pd.to_numeric(df['AgeCategory'], errors='coerce')
age = age.dropna()
age = df['AgeCategory']
kde = gaussian_kde(age)
x = np.linspace(age.min(), age.max(), 12)
kde_values = kde(x)
fig.add_trace(
    go.Histogram(x=age, 
                nbinsx=20, 
                histnorm='probability density', 
                name='AgeCategory', 
                marker=dict(color='orange'),  # Set histogram color to orange
                opacity=0.7  # Add slight transparency to see KDE line
                ),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=x, y=kde_values, mode='lines', name='KDE', line=dict(color='red')),
    row=1, col=2
)


smoking_count = df['Smoking'].value_counts()
# Add "smokers" bar
fig.add_trace(
    go.Bar(
        x=['YES'], 
        y=[smoking_count.get(0, 0)], 
        name='YES',  # Explicit legend label
        marker=dict(color='red'),  # Red for "Did not survive"
        text=[smoking_count.get(0, 0)], 
        textposition='auto'
    ),
    row=2, col=1
)

# Add "NO smokers" bar
fig.add_trace(
    go.Bar(
        x=['NO'], 
        y=[smoking_count.get(1, 0)], 
        name='NO',  # Explicit legend label
        marker=dict(color='blue'),  # Blue for "Survived"
        text=[smoking_count.get(1, 0)], 
        textposition='auto'
    ),
    row=2, col=1
)


# Correlation heatmap with adjusted colorbar
corr_matrixx = df.corr()
corr_matrix=corr_matrixx[['Stroke','AlcoholDrinking','Smoking']]
fig.add_trace(
    go.Heatmap(
        z=corr_matrix.values, 
        x=corr_matrix.columns, 
        y=corr_matrix.columns, 
        colorscale='RdBu',
        text=corr_matrix.values.round(2),  # Add correlation values rounded to 2 decimal places
        texttemplate="%{text}",  # Display the values in each cell
        textfont={"size": 10},  # Set font size for readability
        colorbar=dict(
            len=0.45,  # Set length to match the subplot height (0.4 is roughly the height of one subplot in a 2x2 grid)
            y=0.21,    # Position the colorbar to align with the bottom subplot (row 2)
            yanchor='middle'  # Center the colorbar vertically within the subplot
        )
    ),
    row=2, col=2
)

grouped = df.groupby(['Smoking', 'HeartDisease']).size().unstack(fill_value=0)


fig.add_trace(
    go.Bar(
        name='Sick = 1',
        x=grouped.index,
        y=grouped[1],
        marker_color='red'
    ),    row=3, col=1
)

fig.add_trace(
    go.Bar(
        name='Sick = 0',
        x=grouped.index,
        y=grouped[0],
        marker_color='blue'
    ),    row=3, col=1

)

# Update layout and axes
fig.update_layout(
    height=1200, 
    width=1200, 
    title_text="Univariate and Multivariate Analysis",
    title_x=0.5,  # Center the title
    showlegend=True,
    barmode='group'  # Group the bars for Survival Distribution
)

# Add axis labels for clarity
# fig.update_xaxes(title_text="Survival Status", row=1, col=1)
# fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_xaxes(title_text="Age", row=1, col=2)
fig.update_yaxes(title_text="Probability Density", row=1, col=2)
fig.update_xaxes(title_text="smoking count", row=2, col=1)

return df, fig

df, fig = load_data_and_create_figure()

# App interface
st.title('Heart Disease Predictor')

# Display the interactive Plotly figure
st.subheader('EDA')
st.plotly_chart(fig, use_container_width=False)

# Load pre-trained model and preprocessor
model = joblib.load("project_model.pkl")
preprocessor = joblib.load("preprocessor11.pkl")

# Predictive interface
st.subheader("Model Prediction")
# BMI	Smoking	AlcoholDrinking	Stroke	PhysicalHealth	MentalHealth	DiffWalking	Sex	AgeCategory	Race	Diabetic	PhysicalActivity	GenHealth	SleepTime	Asthma	KidneyDisease	SkinCancer
# Input widgets
col1, col2 = st.columns(2)
with col1:
    BMI = st.number_input("BMI", min_value=12, max_value=87, value=30,step=0.5)
    Smoking = st.selectbox("Smoking", [0,1])
    AlcoholDrinking = st.selectbox("AlcoholDrinking", [0,1])
    Stroke = st.selectbox("Stroke", [0,1])
    PhysicalHealth = st.number_input("PhysicalHealth", min_value=0, max_value=30, value=15)
    MentalHealth = st.number_input("MentalHealth", min_value=0, max_value=30, value=15)
    DiffWalking = st.selectbox("DiffWalking", [0,1])
    Sex = st.selectbox("Sex", [0,1])
   




with col2:
    AgeCategory = st.number_input("AgeCategory", min_value=0, max_value=12, value=10)
    Race = st.selectbox("Race", [0,1])
    Diabetic = st.selectbox("Diabetic", [0,1])
    PhysicalActivity = st.selectbox("PhysicalActivity", [0,1])
    GenHealth = st.number_input("GenHealth", min_value=0, max_value=4, value=2)
    SleepTime = st.number_input("SleepTime", min_value=0, max_value=24, value=10)
    Asthma = st.selectbox("Asthma", [0,1])
    KidneyDisease = st.selectbox("KidneyDisease", [0,1])
    SkinCancer = st.selectbox("SkinCancer", [0,1])

    # Prediction logic
if st.button("Predict case"):
    input_data = pd.DataFrame([[BMI, Smoking, AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,
                                   Race,Diabetic,	PhysicalActivity,	GenHealth,	SleepTime,	Asthma,	KidneyDisease,	SkinCancer]],
                                columns=['BMI', 'Smoking', 'AlcoholDrinking','Stroke','PhysicalHealth','MentalHealth','DiffWalking','Sex','AgeCategory',
                                   'Race',	'Diabetic',	'PhysicalActivity',	'GenHealth',	'SleepTime',	'Asthma',	'KidneyDisease',	'SkinCancer'])

 processed_data = preprocessor11.transform(input_data)
    prediction = xgb.predict(processed_data)[0]
    probability = xgb.predict_proba(processed_data)[0][1]

    st.subheader("Result")
    st.metric("Survival Probability", f"{probability:.1%}")
    st.write(f"Prediction {'Survived' if prediction == 1 else 'Did not survive'}")


Writing st_app_55.py
